Post new ads or change ads one piece at a time

In [1]:
import pandas as pd
import pickle
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException, UnexpectedAlertPresentException, WebDriverException
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.action_chains import ActionChains

YA = 'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/60.0.3112.113 Safari/537.36'
opts = Options()
opts.add_argument("user-agent=YA")
opts.add_argument('--disable-extensions')
opts.add_argument("start-maximized")

In [4]:
df = pd.read_pickle('clist_df')

In [5]:
emails = pickle.load(open('email_info','rb'))
passwords = pickle.load(open('pword_info','rb'))
phone_number = pickle.load(open('phone_info','rb'))
contact_name = pickle.load(open('contact_name','rb'))

In [6]:
# post_title = 'SAT & ACT Expert | 10 yrs Experience | Affordable Rates!' 
# post_title = 'Math & Science Tutor | 10 yrs Experience | Affordable Rates!' 
post_title = 'College Math & Science | 10 yrs Experience | Affordable Rates!' 

In [25]:
def post(loc, email, password):
    '''Login and post to a particular location
    note: user will need to change the first two xpaths to correspond with the specific ad type they would like to post
    note: ad must be copied to clipboard'''
    
    link = df['url'][loc]
    zipcode = df['zip_code'][loc]
    driver = webdriver.Chrome()
    driver.get(link)
    driver.find_element_by_id('post').click()
    driver.find_element_by_xpath('/html/body/article/section/form/ul/li[10]/label/span[2]').click()     #choose ad type
    driver.find_element_by_xpath('/html/body/article/section/form/ul/li[13]/label/span[2]').click()
    
    driver.find_element_by_id('inputEmailHandle').send_keys(email)      #login
    driver.find_element_by_id('inputPassword').send_keys(password)   
    driver.find_element_by_xpath('/html/body/article/section/div/div[1]/form/div[3]/button').click()
    try:
        driver.find_element_by_id('PostingTitle').send_keys(post_title)
    except:           #for those locations with an extra landing page, just choose the first sublocation
        driver.find_element_by_xpath('/html/body/article/section/form/ul/li[1]/label').click()
        try:
            driver.find_element_by_id('PostingTitle').send_keys(post_title)
        except:
            driver.find_element_by_xpath('/html/body/article/section/form/ul/li[2]/label/span[2]').click()
            driver.find_element_by_id('PostingTitle').send_keys(post_title)
    driver.find_element_by_id('postal_code').send_keys(zipcode)
    driver.find_element_by_id('PostingBody').send_keys(Keys.CONTROL, 'v')      # paste ad from clipboard
    
    driver.find_element_by_xpath('//*[@id="contact_phone_ok"]').click()
    driver.find_element_by_xpath('//*[@id="contact_text_ok"]').click()
    driver.find_element_by_xpath('//*[@id="contact_phone"]').send_keys(phone_number)
    driver.find_element_by_xpath('//*[@id="contact_name"]').send_keys(contact_name)
    driver.find_element_by_xpath('//*[@id="wantamap"]').click()
    
    but = driver.find_element_by_css_selector('#postingForm > div > button')     
    ActionChains(driver).send_keys_to_element(but,Keys.ENTER).perform()

    driver.find_element_by_xpath('/html/body/article/section/form/button').click()

#     driver.find_element_by_xpath('//*[@id="publish_top"]/button').click()         #publish
#     ActionChains(driver).send_keys(Keys.CONTROL, 'w').perform()                   #close

In [10]:
locs=list(df.index)

In [11]:
from itertools import cycle
it = cycle(locs)

In [ ]:
for i in range(5):
    post(next(it),emails[i],passwords[i])